In [98]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import ElasticNet
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, GradientBoostingRegressor, StackingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import LabelEncoder
import time
import joblib
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight

In [33]:
df = pd.read_csv('DataSets/pitcher_data_cleaned.csv')

In [34]:
df = df[df['pitch_type']!='PO']

In [110]:
df

,Unnamed: 0,pitch_type,game_date,pitcher,batter,player_name,game_pk,pitch_number,at_bat_number,inning,balls,strikes,zone,outs_when_up,inning_topbot,fld_score,bat_score,on_1b,on_2b,on_3b,if_fielding_alignment,of_fielding_alignment,plate_x,plate_z,pitch_count,prev_pitch_type,score_diff
0,34288,FF,2022-08-21,601713,656775,"Pivetta, Nick",661051,1,4,1,0,0,2.0,0,Bot,0,0,0,0,0,NaN,NaN,0.23,3.04,1,firstPitch,0
1,34287,KC,2022-08-21,601713,656775,"Pivetta, Nick",661051,2,4,1,0,1,12.0,0,Bot,0,0,0,0,0,NaN,NaN,0.71,3.52,2,FF,0
2,34286,FF,2022-08-21,601713,656775,"Pivetta, Nick",661051,3,4,1,1,1,11.0,0,Bot,0,0,0,0,0,NaN,NaN,-0.27,3.80,3,KC,0
3,34285,FF,2022-08-21,601713,656775,"Pivetta, Nick",661051,4,4,1,1,2,11.0,0,Bot,0,0,0,0,0,NaN,NaN,-0.47,3.54,4,FF,0
4,34284,FF,2022-08-21,601713,668939,"Pivetta, Nick",661051,1,5,1,0,0,11.0,0,Bot,0,0,1,0,0,NaN,NaN,-0.74,5.18,5,firstPitch,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48090,48527,FS,2025-02-22,673513,649966,"Matsui, Yuki",779026,1,50,6,0,0,3.0,2,Top,9,2,0,0,0,Standard,Strategic,0.78,2.72,10,firstPitch,7
48091,48526,ST,2025-02-22,673513,649966,"Matsui, Yuki",779026,2,50,6,0,1,12.0,2,Top,9,2,0,0,0,Standard,Strategic,0.68,4.01,11,FS,7
48092,48525,FS,2025-02-22,673513,649966,"Matsui, Yuki",779026,3,50,6,1,1,14.0,2,Top,9,2,0,0,0,Standard,Strategic,0.07,1.36,12,ST,7
48093,48524,FF,2025-02-22,673513,649966,"Matsui, Yuki",779026,4,50,6,1,2,11.0,2,Top,9,2,0,0,0,Standard,Strategic,-1.28,3.02,13,FS,7


In [35]:
df.head()
le = LabelEncoder()

In [36]:
X = df.drop(columns=['pitch_type', 'plate_x', 'plate_z'])
y_pitch_type = le.fit_transform(df['pitch_type'])
y_plate_x = df[['plate_x']]
y_plate_z = df[['plate_z']]

In [111]:
encoded_to_original = dict(zip(le.transform(le.classes_), le.classes_))

In [112]:
encoded_to_original

{0: 'CH',
 1: 'CU',
 2: 'FC',
 3: 'FF',
 4: 'FS',
 5: 'KC',
 6: 'SI',
 7: 'SL',
 8: 'ST'}

preprocessor = ColumnTransformer([
    ('dummify', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ['pitcher', 'batter', 'on_1b', 'on_2b', 'on_3b', 'if_fielding_alignment', 'of_fielding_alignment', 'prev_pitch_type']),
    ("standardize", StandardScaler(), ['inning', 'balls', 'strikes', 'outs_when_up', 'score_diff'])
], remainder='drop')

In [100]:
start = time.time()
preprocessor = ColumnTransformer([
    ('dummify', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ['pitcher', 'batter', 'on_1b', 'on_2b', 'on_3b', 'if_fielding_alignment', 'of_fielding_alignment', 'prev_pitch_type']),
    ("standardize", StandardScaler(), ['inning', 'balls', 'strikes', 'outs_when_up', 'score_diff'])
], remainder='drop')

y_series = pd.Series(y_pitch_type)

classes = np.unique(y_pitch_type)
class_weights = compute_class_weight('balanced', classes=classes, y=y_pitch_type)
class_weight_dict = dict(zip(classes, class_weights))

sample_weights = y_series.map(class_weight_dict).values

XGB = XGBClassifier(n_jobs=-1)

pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('boosting', XGB)
])

param_grid = {
    'boosting__max_depth': [4], 'boosting__n_estimators': [75], 'boosting__learning_rate': [.40]}

stratified_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

grid_search = GridSearchCV(pipeline, param_grid, cv=stratified_cv, scoring='balanced_accuracy', n_jobs=-1)

grid_search.fit(X, y_pitch_type, boosting__sample_weight=sample_weights)
end = time.time()

print(f"Execution time: {end - start:.2f} seconds")
print("Best accuracy score:", grid_search.best_score_)
print("Best parameters:", grid_search.best_params_)

Execution time: 76.36 seconds
Best accuracy score: 0.49074330629358576
Best parameters: {'boosting__learning_rate': 0.4, 'boosting__max_depth': 4, 'boosting__n_estimators': 75}


In [48]:
pitch_type_dict = {'ST':'Sweeper', 'CH': "Changeup", 'FF': "Four Seam Fastball", 'SI': "Sinker", 'SL': "Slider", 'FC': "Cutter", 'CU': "Curveball", 'KC': "Knuckleball", 'FS': "Split-finger"}

In [49]:
pitch_type_dict

{'ST': 'Sweeper',
 'CH': 'Changeup',
 'FF': 'Four Seam Fastball',
 'SI': 'Sinker',
 'SL': 'Slider',
 'FC': 'Cutter',
 'CU': 'Curveball',
 'KC': 'Knuckleball',
 'FS': 'Split-finger'}

In [101]:
joblib.dump(grid_search.best_estimator_, "best_pitch_model.pkl")

['best_pitch_model.pkl']

In [102]:
model = joblib.load('best_pitch_model.pkl')

In [107]:
realtime_input = pd.DataFrame([{
    'pitcher': '605397',
    'batter': '606466',
    'on_1b': 0,   # or np.nan
    'on_2b': 0,
    'on_3b': 0,
    'if_fielding_alignment': '',
    'of_fielding_alignment': '',
    'prev_pitch_type': 'FF',
    'inning': 2,
    'balls': 0,
    'strikes': 2,
    'outs_when_up': 2,
    'score_diff': 0
}])

In [108]:
prediction = model.predict(realtime_input)
pitch_type = le.inverse_transform(prediction)
print("Predicted pitch type:", pitch_type[0])

Predicted pitch type: CH


In [113]:
print(pd.Series(y_pitch_type).value_counts(normalize=True))

3    0.370919
7    0.159916
6    0.121782
0    0.113424
8    0.092444
5    0.043311
2    0.040047
1    0.039714
4    0.018443
dtype: float64
